## Biogas plant

You want to plan the two-year supply of raw materials for a biogas power plant. Such a plant produces energy by burning biogas, which is obtained from the bacterial fermentation of organic wastes.
Specifically, your plant is powered by corn chopping, a residual of agro-industrial operations that you can purchase from 5 local farms.
The table below shows the quarterly capacity of each farm for the next two years. Quantities are measured in tons.

Farm|T1|T2|T3|T4|T5|T6|T7|T8
:-|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:
1|700|1500|700|0|0|700|1500|0
2|1350|0|450|0|1350|0|450|0
3|0|1500|1500|0|0|1500|1500|0
4|820|1560|820|0|820|1560|820|0
5|0|680|1080|0|0|680|1080|0

Due to crop rotations and corn harvesting periods, farms are unable to supply material in some quarters. Moreover the types of corn chopping provided are different, each coming with its own unitary purchase price, unitary storage cost and percentage of dry matter. The table below shows a summary of these information.

Farm|Purchase price|Storage cost|Dry matter
:-|:-:|:-:|:-:
1|0.20|0.002|15
2|0.18|0.012|28
3|0.19|0.007|35
4|0.21|0.011|37
5|0.23|0.015|42

Your biogas plant must operate by burning a mixture of corn choppings with a dry matter percentage between 20% and 40%. Under these conditions, the yield is 421.6 kWh of energy per ton of burned material. The energy produced by the plant is sold on the market at a price of 0.28 $/kWh.

Due to state regulations, all biogas plants can produce a maximum of 1950 MWh of energy per quarter. You are allowed to store corn chopping in a silo, whose total capacity is of 500 tons.

Plan the supply and inventory of your biogas plant with the goal of maximizing your profits (i.e., revenues minus costs).

In [ ]:
# When using Colab, make sure you run this instruction beforehand
!pip install --upgrade cffi==1.15.0
import importlib
import cffi
importlib.reload(cffi)
!pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 446 kB 13.8 MB/s 
  Attempting uninstall: cffi
    Found existing installation: cffi 1.15.1
    Uninstalling cffi-1.15.1:
      Successfully uninstalled cffi-1.15.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.3 MB 4.3 MB/s 


# Biogas Plant resolution


In order to model de problem, it was defined the following sets, variables, constraints and objective function in order to make it simpler to apply what was described in class.

----------------------
## 1 - Sets

Given the problem, two sets are definable to iterate over each quarter and for each farm.

$F = \{ 1,2,3,4,5 \} → \text{set of farms in the problem}$

$Q = \{ 1,2,3,4,5, 6, 7, 8 \} → \text{quarters comprehended in the problem}$

----------------------
## 2 - Variables

In order to apply the constraints and iterate over the sets, there were two variables defined. Both of them were defined from 0 to 1 to show relative proportions.

$x_{ij} → \text{total production effectivelly used from farm i at a quarter j} $

$s_{ij} → \text{quantity that was bought from farm i at a quarter j, before consumption} $

$silo_{i, j} → \text{quantity stored in the silo, for each farm i in quarter j}$

----------------------
## 3 - Constraints

There were 5 main points in the problem that created the constraint equations, they were:

### 3.1 Maximum Material by Farm

In order to ensure that it is not possible to buy more quantity than the one offered by each farm, a constraint is needed for the maximum purchase, given by, for each $i \in F$ and $j \in Q$:

$$
s_{ij} \leqslant A_{ij}
$$

$A_{ij} → \text{Quarterly capacity from farm i in quarter j}$

### 3.2 Energy Produced by Quarter

That generates 8 constraints (one for each of the quarters in j):

$$ 421.6 \sum_{i \in F} x_{ij} \leqslant 1950 × 10^3 $$



### 3.3 Storage Limit

This limit also generates 8 constraints (one for each of the 8 quarters established):

$$ \sum_{i \in F} s_{ij}-x_{ij} + silo_{j+1} \leqslant 500 $$

$ silo_{j-1} \rightarrow \text{previous storage quantity in the silo (described in a clear way in the code)} $

### 3.4 Dry Material

Thia condition generates 16 equations (8 to guarantee the below 40% ration, and 8 more to guarantee the above 20%):

For each $j \in Q$:

$$15x_{1j} + 28x_{2j} + 35x_{3j} + 37x_{4j} + 42x_{5j} \leqslant 40\sum_{i \in F} x_{ij} $$

$$15x_{1j} + 28x_{2j} + 35x_{3j} + 37x_{4j} + 42x_{5j} \geqslant 20\sum_{i \in F} x_{ij} $$

### 3.5 Storage constraint

This last constraint refers to the fact that the storage of not used material in the silo for the next quarter is the remaining storage plus the difference between bought material and used material from the current quarter

For each $i \in F$ and $j \in Q$:

$$
silo_{i, j+1} = silo_{ij} + (s_{ij} - x_{ij})  
$$

----------------------
## 4 Objective Function

At last. the objective function can be described as the attempt to maximize the profit from the sum of all quarters:

$$
\text{Objective Function → max (R - C)}
$$

$$
R = 0.28 \times 421.6 \times \sum_{j \in Q}\sum_{i \in F}x_{ij}
$$

$$
C = \sum_{j \in Q} (\sum_{i \in F} \alpha_i \cdot s_{ij} + \sum_{i \in F} \beta_i (silo_{i, j+1}))
$$


$\alpha_i → \text{purchase cost from farm i} $

$\beta_i → \text{storage cost from farm i} $

In [ ]:
##################################### CONSTANT DECLARATION

import mip

m = mip.Model()

A = [
    [ 700, 1500,  700, 0,    0,  700, 1500,  0],
    [1350,    0,  450, 0, 1350,    0,  450,  0],
    [   0, 1500, 1500, 0,	   0,	1500,	1500,	 0],
    [ 820, 1560,	820, 0,	 820,	1560,	 820,	 0],
    [   0,	680, 1080, 0,	   0,  680,	1080,	 0]
    ]

n_farm = len(A)     #len of set F
n_quart = len(A[0]) #len of set Q

c = [
    [ 0.20,  0.18,  0.19,  0.21,   0.23], #Purchase price
    [0.002, 0.012, 0.007, 0.011, 0.0155], #storage price
    [ 0.15,  0.28,  0.35,  0.37,   0.42]  #Dry matter percentage
]

pp = c[0] #Purchase price
sp = c[1] #storage price
dmp = c[2] #Dry matter percentage

max_epq = 1950000 # max kWh produced per quarter
max_tpq = 500 # max ton stored per quarter
max_dmp = 0.4 # max dry matter percentage
min_dmp = 0.2 # min dry matter percentage

kWh_per_ton = 421.6 #kWh/ton
price_per_kWh = 0.28 #$/kWh
price_per_ton = kWh_per_ton * price_per_kWh #$/ton

In [ ]:
##################################### VARIABLE DECLARATION

# create var to store the quantity of tons inside the silo
silo = [[m.add_var(var_type=mip.INTEGER) for j in range(n_farm)] for i in range(n_quart)]
silo.insert(0, [0. for _ in range(n_farm)])

# each link represents the weight produce in each farm each quarter = link
x = [[m.add_var(var_type=mip.INTEGER) for i in range(n_quart)] for j in range(n_farm)] # quantity burned
s = [[m.add_var(var_type=mip.INTEGER) for i in range(n_quart)] for j in range(n_farm)] # quantity bought

In [ ]:
##################################### CONSTRAINT ITERATION

for q in range(n_quart):

    # The maximum space inside the silo is 500 tons.
    m.add_constr(mip.xsum(silo[q+1]) <= max_tpq)

    # Between 20% e 40% of dry matter
    total_dmp = mip.xsum(x[f][q] * dmp[f] for f in range(n_farm))
    x_total = mip.xsum(x[f][q] for f in range(n_farm))
    m.add_constr(total_dmp <= max_dmp*x_total)
    m.add_constr(total_dmp >= min_dmp*x_total)

    # Energy production limit
    m.add_constr(mip.xsum(x[f][q] for f in range(n_farm))*kWh_per_ton <= max_epq)

for f in range(n_farm):
  for q in range(n_quart):
    # Just buy what is available
    m.add_constr(s[f][q] <= A[f][q])
    # Stores into the silo material not used.
    m.add_constr((silo[q+1][f]) == silo[q][f] + (s[f][q] - x[f][q]))

In [ ]:
##################################### OBJECTIVE FUNCTION

m.objective = mip.maximize(mip.xsum(x[f][q] for f in range(n_farm) for q in range(n_quart)) * price_per_ton - mip.xsum(s[f][q]*pp[f] for f in range(n_farm) for q in range(n_quart)) - mip.xsum(silo[q+1][f]*sp[f] for f in range(n_farm) for q in range(n_quart)))

m.optimize()

print("Maximum revenue:", m.objective_value)

profit 2861318.0329999994


In [ ]:
##################################### VISUALIZATION OF RESULTS

print("limit per quarter {0} kWh ; {1} tons in stock ; {2}% to {3}% of dry matter \n".format(max_epq,max_tpq,min_dmp*100,max_dmp*100))

for q in range(n_quart):
  xx = sum(x[f][q].x for f in range(n_farm))
  ss = sum(s[f][q].x for f in range(n_farm))
  ssilo = sum(silo[q+1][f].x for f in range(n_farm))

  dm = sum(x[f][q].x*dmp[f] for f in range(n_farm))
  dm_tot = sum(x[f][q].x for f in range(n_farm))
  dm_per= 0
  if dm_tot>0:
    dm_per = dm*100/dm_tot
  print("QUARTER {0}    produced: {1:4.2f} ton    stocked: {2:.2f}   bought:{3:.2f} ton   dry matter: {4:.2f}%".format(q, xx, ssilo, ss, dm_per))
  print("                       {0:4.2f} MWh ".format( xx*kWh_per_ton/1e3 ))
  print("                       {0:4.2f}$\n".format(xx*price_per_ton))

limit per quarter 1950000 kWh ; 500 tons in stock ; 20.0% to 40.0% of dry matter 

QUARTER 0    produced: 2870.00 ton    stocked: 0.00   bought:2870.00 ton   dry matter: 27.40%
                       1209.99 MWh 
                       338797.76$

QUARTER 1    produced: 4625.00 ton    stocked: 500.00   bought:5125.00 ton   dry matter: 32.21%
                       1949.90 MWh 
                       545972.00$

QUARTER 2    produced: 4625.00 ton    stocked: 425.00   bought:4550.00 ton   dry matter: 32.50%
                       1949.90 MWh 
                       545972.00$

QUARTER 3    produced: 424.00 ton    stocked: 1.00   bought:0.00 ton   dry matter: 20.00%
                       178.76 MWh 
                       50052.35$

QUARTER 4    produced: 2171.00 ton    stocked: 0.00   bought:2170.00 ton   dry matter: 31.39%
                       915.29 MWh 
                       256282.21$

QUARTER 5    produced: 4440.00 ton    stocked: 0.00   bought:4440.00 ton   dry matter: 33.62%
 